### A Comparative Study of Lambda Setting

#### Necessary Imports

In [14]:
import numpy as np

import torch

#### Lambda

In [13]:
def ideal_dcg(ideally_sorted_labels):
    '''

    :param ideally_sorted_labels:
    :return:
    '''
    gains = np.power(2.0, ideally_sorted_labels) - 1.0
    ranks = np.arange(len(ideally_sorted_labels)) + 1.0

    discounts = np.log2(1.0 + ranks)
    ideal_dcg = np.sum(gains / discounts)

    return ideal_dcg

def get_delta_gains(labels_sorted_via_preds):
    gains = np.power(2.0, labels_sorted_via_preds) - 1.0
    gain_diffs = np.expand_dims(gains, axis=1) - np.expand_dims(gains, axis=0)
    delta_gain = np.abs(gain_diffs)  # absolute delta gains w.r.t. pairwise swapping

    return delta_gain

def get_delta_ndcg(ideally_sorted_labels, labels_sorted_via_preds):
    '''
    Delta-nDCG w.r.t. pairwise swapping of the currently predicted ranking
    '''
    idcg = ideal_dcg(ideally_sorted_labels) # ideal discount cumulative gains

    gains = np.power(2.0, labels_sorted_via_preds) - 1.0
    n_gains = gains / idcg                  # normalised gains
    ng_diffs = np.expand_dims(n_gains, axis=1) - np.expand_dims(n_gains, axis=0)

    ranks = np.arange(len(labels_sorted_via_preds)) + 1.0
    dists = 1.0 / np.log2(ranks + 1.0)      # discount co-efficients
    dists_diffs = np.expand_dims(dists, axis=1) - np.expand_dims(dists, axis=0)
    mat_delta_ndcg = np.abs(ng_diffs) * np.abs(dists_diffs)  # absolute changes w.r.t. pairwise swapping

    return mat_delta_ndcg

def to_gain(labels):
    return np.power(2.0, labels) - 1.0

#### Delta-nDCG

In [12]:
ideally_sorted_labels   = np.asarray([2, 2, 2, 1, 1, 0])
labels_sorted_via_preds = np.asarray([0, 2, 1, 2, 1, 2])


mat_delta_ndcg = get_delta_ndcg(ideally_sorted_labels, labels_sorted_via_preds)
print('delta-nDCG:\n', mat_delta_ndcg)

delta-nDCG:
 [[0.         0.1535592  0.06934506 0.23687862 0.08503746 0.26786312]
 [0.1535592  0.         0.03631733 0.         0.06770212 0.        ]
 [0.06934506 0.03631733 0.         0.01922895 0.         0.03988529]
 [0.23687862 0.         0.01922895 0.         0.01215584 0.        ]
 [0.08503746 0.06770212 0.         0.01215584 0.         0.00850049]
 [0.26786312 0.         0.03988529 0.         0.00850049 0.        ]]


In [79]:
mat_delta_ndcg = get_delta_ndcg(ideally_sorted_labels, ideally_sorted_labels)
print('delta-nDCG:\n', mat_delta_ndcg)

delta-nDCG:
 [[0.         0.         0.         0.15791908 0.17007492 0.26786312]
 [0.         0.         0.         0.05554628 0.06770212 0.11430392]
 [0.         0.         0.         0.01922895 0.0313848  0.05982793]
 [0.15791908 0.05554628 0.01922895 0.         0.         0.01032817]
 [0.17007492 0.06770212 0.0313848  0.         0.         0.00425024]
 [0.26786312 0.11430392 0.05982793 0.01032817 0.00425024 0.        ]]


In [11]:
delta_gain = get_delta_gains(labels_sorted_via_preds)
print('delta-gain:\n', delta_gain)

delta-gain:
 [[0. 3. 1. 3. 1. 3.]
 [3. 0. 2. 0. 2. 0.]
 [1. 2. 0. 2. 0. 2.]
 [3. 0. 2. 0. 2. 0.]
 [1. 2. 0. 2. 0. 2.]
 [3. 0. 2. 0. 2. 0.]]
